# Anomaly Detection Model

## Skripsi - Hagen Kwik

In [1]:
# library for mathematical functions and data preprocessing such as table, etc
import pandas as pd
import numpy as np
import pickle

# Library for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

# Library for machine learning functions/algorithms
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import SGD, Nadam, Adam

# Use the legacy Adam optimizer
from mealpy import WOA, DE
from mealpy.utils.problem import FloatVar

#  Library for evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # Set memory growth for GPU (to avoid full memory allocation)
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("Memory growth enabled for GPUs")
    except RuntimeError as e:
        print(e)

## Setup

### KDD99 Dataset

In [3]:
# data
df_kdd99_training = pd.read_csv('kdd99-training.csv')
df_kdd99_training.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal


In [4]:
df_kdd99_testing = pd.read_csv('kdd99-testing.csv')
df_kdd99_testing.head()


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.00,0.00,0.03,0.03,back.
1,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.01,0.01,0.04,0.04,back.
2,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.00,0.00,0.03,0.03,back.
3,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.00,0.00,0.02,0.02,back.
4,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.01,0.01,0.05,0.05,back.


In [5]:
df_kdd99_training.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,494020.000000,4.940200e+05,4.940200e+05,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,...,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000
mean,47.979400,3.025616e+03,8.685308e+02,0.000045,0.006433,0.000014,0.034519,0.000152,0.148245,0.010212,...,232.471248,188.666052,0.753781,0.030906,0.601936,0.006684,0.176754,0.176443,0.058118,0.057412
std,707.747185,9.882191e+05,3.304003e+04,0.006673,0.134805,0.005510,0.782103,0.015520,0.355343,1.798328,...,64.744601,106.040205,0.410780,0.109259,0.481309,0.042133,0.380593,0.380920,0.230590,0.230141
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,46.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,884.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
df_kdd99_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494020 entries, 0 to 494019
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     494020 non-null  int64  
 1   protocol_type                494020 non-null  object 
 2   service                      494020 non-null  object 
 3   flag                         494020 non-null  object 
 4   src_bytes                    494020 non-null  int64  
 5   dst_bytes                    494020 non-null  int64  
 6   land                         494020 non-null  int64  
 7   wrong_fragment               494020 non-null  int64  
 8   urgent                       494020 non-null  int64  
 9   hot                          494020 non-null  int64  
 10  num_failed_logins            494020 non-null  int64  
 11  logged_in                    494020 non-null  int64  
 12  num_compromised              494020 non-null  int64  
 13 

In [7]:
df_kdd99_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311028 entries, 0 to 311027
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     311028 non-null  int64  
 1   protocol_type                311028 non-null  object 
 2   service                      311028 non-null  object 
 3   flag                         311028 non-null  object 
 4   src_bytes                    311028 non-null  int64  
 5   dst_bytes                    311028 non-null  int64  
 6   land                         311028 non-null  int64  
 7   wrong_fragment               311028 non-null  int64  
 8   urgent                       311028 non-null  int64  
 9   hot                          311028 non-null  int64  
 10  num_failed_logins            311028 non-null  int64  
 11  logged_in                    311028 non-null  int64  
 12  num_compromised              311028 non-null  int64  
 13 

In [8]:
# Check which column needs fixing
for column in df_kdd99_testing.columns:
    unique_values = df_kdd99_testing[column].unique()
    print(f"Unique values in '{column}':")
    print(unique_values)
    print()

Unique values in 'duration':
[    0     9     2 ...  5049 10121  5033]

Unique values in 'protocol_type':
['tcp' 'icmp' 'udp']

Unique values in 'service':
['http' 'telnet' 'login' 'imap4' 'eco_i' 'private' 'smtp' 'whois' 'ssh'
 'ctf' 'ecr_i' 'domain' 'ftp' 'gopher' 'finger' 'rje' 'time' 'uucp' 'efs'
 'other' 'vmnet' 'klogin' 'netbios_ns' 'pop_2' 'netstat' 'remote_job'
 'shell' 'hostnames' 'ftp_data' 'http_443' 'netbios_ssn' 'iso_tsap'
 'csnet_ns' 'link' 'ldap' 'supdup' 'echo' 'pop_3' 'courier' 'name' 'nnsp'
 'exec' 'printer' 'systat' 'daytime' 'bgp' 'sql_net' 'auth' 'Z39_50' 'mtp'
 'kshell' 'uucp_path' 'netbios_dgm' 'sunrpc' 'nntp' 'discard' 'domain_u'
 'urp_i' 'IRC' 'ntp_u' 'urh_i' 'X11' 'tim_i' 'harvest']

Unique values in 'flag':
['SF' 'RSTR' 'RSTO' 'REJ' 'S0' 'SH' 'S1' 'S3' 'S2' 'OTH' 'RSTOS0']

Unique values in 'src_bytes':
[  54540   53168   54020 ...     520    1237 5133876]

Unique values in 'dst_bytes':
[     8314      7300      1460 ...      6944      4850 400291060]

Unique

In [9]:
# Check which column needs fixing
for column in df_kdd99_training.columns:
    unique_values = df_kdd99_training[column].unique()
    print(f"Unique values in '{column}':")
    print(unique_values)
    print()

Unique values in 'duration':
[   0    1   79 ... 2695 2751  120]

Unique values in 'protocol_type':
['tcp' 'udp' 'icmp']

Unique values in 'service':
['http' 'smtp' 'finger' 'domain_u' 'auth' 'telnet' 'ftp' 'eco_i' 'ntp_u'
 'ecr_i' 'other' 'private' 'pop_3' 'ftp_data' 'rje' 'time' 'mtp' 'link'
 'remote_job' 'gopher' 'ssh' 'name' 'whois' 'domain' 'login' 'imap4'
 'daytime' 'ctf' 'nntp' 'shell' 'IRC' 'nnsp' 'http_443' 'exec' 'printer'
 'efs' 'courier' 'uucp' 'klogin' 'kshell' 'echo' 'discard' 'systat'
 'supdup' 'iso_tsap' 'hostnames' 'csnet_ns' 'pop_2' 'sunrpc' 'uucp_path'
 'netbios_ns' 'netbios_ssn' 'netbios_dgm' 'sql_net' 'vmnet' 'bgp' 'Z39_50'
 'ldap' 'netstat' 'urh_i' 'X11' 'urp_i' 'pm_dump' 'tftp_u' 'tim_i' 'red_i']

Unique values in 'flag':
['SF' 'S1' 'REJ' 'S2' 'S0' 'S3' 'RSTO' 'RSTR' 'RSTOS0' 'OTH' 'SH']

Unique values in 'src_bytes':
[181 239 235 ... 500 484 475]

Unique values in 'dst_bytes':
[ 5450   486  1337 ... 34557 10592 13828]

Unique values in 'land':
[0 1]

Unique valu

In [10]:
missing_values_kdd99_testing = df_kdd99_testing.isnull().sum()
print(missing_values_kdd99_testing)

duplicates_kdd99_testing = df_kdd99_testing.duplicated().sum()
print(duplicates_kdd99_testing)


duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

In [11]:
missing_values_kdd99_training = df_kdd99_training.isnull().sum()
print(missing_values_kdd99_training)

duplicates_kdd99_training = df_kdd99_training.duplicated().sum()
print(duplicates_kdd99_training)


duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

### UNSW-NB15 Dataset

In [12]:
# data
df_unsw_training = pd.read_csv('UNSW-training-set.csv')
df_unsw_testing = pd.read_csv('UNSW-testing-set.csv')
df_unsw_training.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [13]:
df_unsw_testing.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [14]:
# Check which column needs fixing
df_unsw_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175341 entries, 0 to 175340
Data columns (total 45 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 175341 non-null  int64  
 1   dur                175341 non-null  float64
 2   proto              175341 non-null  object 
 3   service            175341 non-null  object 
 4   state              175341 non-null  object 
 5   spkts              175341 non-null  int64  
 6   dpkts              175341 non-null  int64  
 7   sbytes             175341 non-null  int64  
 8   dbytes             175341 non-null  int64  
 9   rate               175341 non-null  float64
 10  sttl               175341 non-null  int64  
 11  dttl               175341 non-null  int64  
 12  sload              175341 non-null  float64
 13  dload              175341 non-null  float64
 14  sloss              175341 non-null  int64  
 15  dloss              175341 non-null  int64  
 16  si

In [15]:
# Check which column needs fixing
df_unsw_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 82332 non-null  int64  
 1   dur                82332 non-null  float64
 2   proto              82332 non-null  object 
 3   service            82332 non-null  object 
 4   state              82332 non-null  object 
 5   spkts              82332 non-null  int64  
 6   dpkts              82332 non-null  int64  
 7   sbytes             82332 non-null  int64  
 8   dbytes             82332 non-null  int64  
 9   rate               82332 non-null  float64
 10  sttl               82332 non-null  int64  
 11  dttl               82332 non-null  int64  
 12  sload              82332 non-null  float64
 13  dload              82332 non-null  float64
 14  sloss              82332 non-null  int64  
 15  dloss              82332 non-null  int64  
 16  sinpkt             823

In [16]:
# Check which column needs fixing
for column in df_unsw_training.columns:
    unique_values = df_unsw_training[column].unique()
    print(f"Unique values in '{column}':")
    print(unique_values)
    print()
    

Unique values in 'id':
[     1      2      3 ... 175339 175340 175341]

Unique values in 'dur':
[0.121478 0.649902 1.623129 ... 3.71911  0.996503 1.557125]

Unique values in 'proto':
['tcp' 'udp' 'arp' 'ospf' 'icmp' 'igmp' 'rtp' 'ddp' 'ipv6-frag' 'cftp'
 'wsn' 'pvp' 'wb-expak' 'mtp' 'pri-enc' 'sat-mon' 'cphb' 'sun-nd' 'iso-ip'
 'xtp' 'il' 'unas' 'mfe-nsp' '3pc' 'ipv6-route' 'idrp' 'bna' 'swipe'
 'kryptolan' 'cpnx' 'rsvp' 'wb-mon' 'vmtp' 'ib' 'dgp' 'eigrp' 'ax.25'
 'gmtp' 'pnni' 'sep' 'pgm' 'idpr-cmtp' 'zero' 'rvd' 'mobile' 'narp' 'fc'
 'pipe' 'ipcomp' 'ipv6-no' 'sat-expak' 'ipv6-opts' 'snp' 'ipcv'
 'br-sat-mon' 'ttp' 'tcf' 'nsfnet-igp' 'sprite-rpc' 'aes-sp3-d' 'sccopmce'
 'sctp' 'qnx' 'scps' 'etherip' 'aris' 'pim' 'compaq-peer' 'vrrp' 'iatp'
 'stp' 'l2tp' 'srp' 'sm' 'isis' 'smp' 'fire' 'ptp' 'crtp' 'sps'
 'merit-inp' 'idpr' 'skip' 'any' 'larp' 'ipip' 'micp' 'encap' 'ifmp'
 'tp++' 'a/n' 'ipv6' 'i-nlsp' 'ipx-n-ip' 'sdrp' 'tlsp' 'gre' 'mhrp' 'ddx'
 'ippc' 'visa' 'secure-vmtp' 'uti' 'vines

In [17]:
# Check for null values
missing_values_unsw_training = df_unsw_training.isnull().sum()
print(missing_values_unsw_training)

# Check for duplicate rows
duplicate_unsw_training = df_unsw_training.duplicated().sum()
print(duplicate_unsw_training)

id                   0
dur                  0
proto                0
service              0
state                0
spkts                0
dpkts                0
sbytes               0
dbytes               0
rate                 0
sttl                 0
dttl                 0
sload                0
dload                0
sloss                0
dloss                0
sinpkt               0
dinpkt               0
sjit                 0
djit                 0
swin                 0
stcpb                0
dtcpb                0
dwin                 0
tcprtt               0
synack               0
ackdat               0
smean                0
dmean                0
trans_depth          0
response_body_len    0
ct_srv_src           0
ct_state_ttl         0
ct_dst_ltm           0
ct_src_dport_ltm     0
ct_dst_sport_ltm     0
ct_dst_src_ltm       0
is_ftp_login         0
ct_ftp_cmd           0
ct_flw_http_mthd     0
ct_src_ltm           0
ct_srv_dst           0
is_sm_ips_ports      0
attack_cat 

In [18]:
# Check for null values
missing_values_unsw_testing = df_unsw_testing.isnull().sum()
print(missing_values_unsw_testing)

# Check for duplicate rows
duplicate_unsw_testing = df_unsw_testing.duplicated().sum()
print(duplicate_unsw_testing)

id                   0
dur                  0
proto                0
service              0
state                0
spkts                0
dpkts                0
sbytes               0
dbytes               0
rate                 0
sttl                 0
dttl                 0
sload                0
dload                0
sloss                0
dloss                0
sinpkt               0
dinpkt               0
sjit                 0
djit                 0
swin                 0
stcpb                0
dtcpb                0
dwin                 0
tcprtt               0
synack               0
ackdat               0
smean                0
dmean                0
trans_depth          0
response_body_len    0
ct_srv_src           0
ct_state_ttl         0
ct_dst_ltm           0
ct_src_dport_ltm     0
ct_dst_sport_ltm     0
ct_dst_src_ltm       0
is_ftp_login         0
ct_ftp_cmd           0
ct_flw_http_mthd     0
ct_src_ltm           0
ct_srv_dst           0
is_sm_ips_ports      0
attack_cat 

## EDA & Preprocess

### Remove Duplicates

#### KDD99

In [19]:
print(f"Testing")
print(f"Number of rows before removing duplicates: {len(df_kdd99_testing)}")
# Remove duplicates and overwrite the original DataFrame
df_kdd99_testing = df_kdd99_testing.drop_duplicates().reset_index(drop=True)
# Check number of rows after removing duplicates
print(f"Number of rows after removing duplicates: {len(df_kdd99_testing)}")
# Check if any duplicates remain
remaining_duplicates_df_kdd99_testing = df_kdd99_testing.duplicated().sum()
print(f"Number of duplicate rows remaining: {remaining_duplicates_df_kdd99_testing}")


print(f"Training")
print(f"Number of rows before removing duplicates: {len(df_kdd99_training)}")
# Remove duplicates and overwrite the original DataFrame
df_kdd99_training = df_kdd99_training.drop_duplicates().reset_index(drop=True)
# Check number of rows after removing duplicates
print(f"Number of rows after removing duplicates: {len(df_kdd99_training)}")
# Check if any duplicates remain
remaining_duplicates = df_kdd99_training.duplicated().sum()
print(f"Number of duplicate rows remaining: {remaining_duplicates}")


Testing
Number of rows before removing duplicates: 311028
Number of rows after removing duplicates: 97325
Number of duplicate rows remaining: 0
Training
Number of rows before removing duplicates: 494020
Number of rows after removing duplicates: 145585
Number of duplicate rows remaining: 0


#### UNSW

In [20]:
print(f"Testing")
print(f"Number of rows before removing duplicates: {len(df_unsw_testing)}")
# Remove duplicates and overwrite the original DataFrame
df_unsw_testing = df_unsw_testing.drop_duplicates().reset_index(drop=True)
# Check number of rows after removing duplicates
print(f"Number of rows after removing duplicates: {len(df_unsw_testing)}")
# Check if any duplicates remain
remaining_duplicates_df_unsw_testing = df_unsw_testing.duplicated().sum()
print(f"Number of duplicate rows remaining: {remaining_duplicates_df_unsw_testing}")


print(f"Training")
print(f"Number of rows before removing duplicates: {len(df_unsw_training)}")
# Remove duplicates and overwrite the original DataFrame
df_unsw_training = df_unsw_training.drop_duplicates().reset_index(drop=True)
# Check number of rows after removing duplicates
print(f"Number of rows after removing duplicates: {len(df_unsw_training)}")
# Check if any duplicates remain
remaining_duplicates = df_unsw_training.duplicated().sum()
print(f"Number of duplicate rows remaining: {remaining_duplicates}")

Testing
Number of rows before removing duplicates: 82332
Number of rows after removing duplicates: 82332
Number of duplicate rows remaining: 0
Training
Number of rows before removing duplicates: 175341
Number of rows after removing duplicates: 175341
Number of duplicate rows remaining: 0


### Label Encoding

#### KDD99

In [21]:
# Initialize encoders
protocol_encoder = LabelEncoder()
service_encoder = LabelEncoder()
flag_encoder = LabelEncoder()

# Fit encoders on training data (with 'others' added)
df_kdd99_training['protocol_type'] = protocol_encoder.fit_transform(df_kdd99_training['protocol_type'])
df_kdd99_training['service'] = service_encoder.fit_transform(df_kdd99_training['service'])
df_kdd99_training['flag'] = flag_encoder.fit_transform(df_kdd99_training['flag'])

# Function to replace unseen categories with 'others'
def handle_unseen(data, encoder, column):
    known_categories = set(encoder.classes_)
    data[column] = data[column].apply(lambda x: x if x in known_categories else 'others')
    return data

# Apply to test data
df_kdd99_testing = handle_unseen(df_kdd99_testing, protocol_encoder, 'protocol_type')
df_kdd99_testing = handle_unseen(df_kdd99_testing, service_encoder, 'service')
df_kdd99_testing = handle_unseen(df_kdd99_testing, flag_encoder, 'flag')

# Add 'others' to encoder classes if not present (optional)
for encoder in [protocol_encoder, service_encoder, flag_encoder]:
    if 'others' not in encoder.classes_:
        encoder.classes_ = np.append(encoder.classes_, 'others')

# Transform test data
df_kdd99_testing['protocol_type'] = protocol_encoder.transform(df_kdd99_testing['protocol_type'])
df_kdd99_testing['service'] = service_encoder.transform(df_kdd99_testing['service'])
df_kdd99_testing['flag'] = flag_encoder.transform(df_kdd99_testing['flag'])

#### UNSW

In [22]:
from sklearn.preprocessing import LabelEncoder

# Initialize encoders
state_encoder = LabelEncoder()
service_encoder = LabelEncoder()
proto_encoder = LabelEncoder()

# Fit and transform training data
df_unsw_training['state'] = state_encoder.fit_transform(df_unsw_training['state'])
df_unsw_training['service'] = service_encoder.fit_transform(df_unsw_training['service'])
df_unsw_training['proto'] = proto_encoder.fit_transform(df_unsw_training['proto'])

# Handle unseen categories in test data by mapping to 'INT'
def handle_unseen(data, encoder, column, default='INT'):
    # Get known categories from encoder
    known_categories = set(encoder.classes_)
    
    # Replace unseen values with default
    data[column] = data[column].apply(lambda x: x if x in known_categories else default)
    
    # Add default to encoder if not present
    if default not in encoder.classes_:
        encoder.classes_ = np.append(encoder.classes_, default)
    
    return data

# Apply to test data
df_unsw_testing = handle_unseen(df_unsw_testing, state_encoder, 'state', 'INT')
df_unsw_testing = handle_unseen(df_unsw_testing, service_encoder, 'service', '-')
df_unsw_testing = handle_unseen(df_unsw_testing, proto_encoder, 'proto', 'udp')

# Transform test data
df_unsw_testing['state'] = state_encoder.transform(df_unsw_testing['state'])
df_unsw_testing['service'] = service_encoder.transform(df_unsw_testing['service'])
df_unsw_testing['proto'] = proto_encoder.transform(df_unsw_testing['proto'])

### Drop column

In [23]:
columns_to_drop_unsw = ['attack_cat', 'id', 'label']

# training
df_autoencoder_unsw_train = df_unsw_training[df_unsw_training['label'] == 0].drop(columns=columns_to_drop_unsw)

df_unsw_training_label = df_unsw_training['label']
df_unsw_training = df_unsw_training.drop(columns=columns_to_drop_unsw)

In [24]:
columns_to_drop_kdd = ['label']

# training
df_autoencoder_kdd99_train = df_kdd99_training[df_kdd99_training['label'] == 'normal'].drop(columns=columns_to_drop_kdd)

df_kdd99_training_label = df_kdd99_training['label']
df_kdd99_training = df_kdd99_training.drop(columns=columns_to_drop_kdd)

In [ ]:
columns_to_drop_unsw = ['attack_cat', 'id', 'label']

df_autoencoder_unsw_train = df_unsw_training[df_unsw_training['label'] == 0].drop(columns=columns_to_drop_unsw)
df_unsw_training_label = df_unsw_training['label']
df_unsw_training = df_unsw_training.drop(columns=columns_to_drop_unsw)

columns_to_drop_kdd = ['label']
df_autoencoder_kdd99_train = df_kdd99_training[df_kdd99_training['label'] == 'normal'].drop(columns=columns_to_drop_kdd)
df_kdd99_training_label = df_kdd99_training['label']
df_kdd99_training = df_kdd99_training.drop(columns=columns_to_drop_kdd)

### Correlation

In [25]:
spearman_corr_unsw = df_unsw_training.corr(method='spearman').abs()

# Create a set to store columns to keep and drop
cols_to_keep = set(spearman_corr_unsw.columns)
cols_to_drop = set()

# Iterate through the correlation matrix
for i in range(len(spearman_corr_unsw.columns)):
    for j in range(i):
        if spearman_corr_unsw.iloc[i, j] > 0.8:  # Check if correlation > 0.5
            col_i = spearman_corr_unsw.columns[i]
            col_j = spearman_corr_unsw.columns[j]
            
            # Print the correlated pair
            original_corr = spearman_corr_unsw.loc[col_i, col_j]
            print(f"Highly correlated pair: {col_i} - {col_j} (r = {original_corr:.2f})")
            
            # Keep the first one and drop the second (you can modify this logic)
            if col_j in cols_to_keep:
                cols_to_drop.add(col_i)
                print(f"  → Keeping '{col_j}', dropping '{col_i}'")

# Convert to lists
cols_to_keep = list(cols_to_keep - cols_to_drop)
cols_to_drop = list(cols_to_drop)

# Create filtered dataframe
df_unsw_filtered_spearman = spearman_corr_unsw[cols_to_keep]

# Print summary
print("\nVariables kept:", cols_to_keep)
print("Variables dropped:", cols_to_drop)
print("\nOriginal shape:", spearman_corr_unsw.shape)
print("Filtered shape:", df_unsw_filtered_spearman.shape)

Highly correlated pair: spkts - dur (r = 0.82)
  → Keeping 'dur', dropping 'spkts'
Highly correlated pair: dpkts - state (r = 0.82)
  → Keeping 'state', dropping 'dpkts'
Highly correlated pair: dpkts - spkts (r = 0.91)
  → Keeping 'spkts', dropping 'dpkts'
Highly correlated pair: sbytes - spkts (r = 0.90)
  → Keeping 'spkts', dropping 'sbytes'
Highly correlated pair: sbytes - dpkts (r = 0.81)
  → Keeping 'dpkts', dropping 'sbytes'
Highly correlated pair: dbytes - state (r = 0.83)
  → Keeping 'state', dropping 'dbytes'
Highly correlated pair: dbytes - spkts (r = 0.89)
  → Keeping 'spkts', dropping 'dbytes'
Highly correlated pair: dbytes - dpkts (r = 0.99)
  → Keeping 'dpkts', dropping 'dbytes'
Highly correlated pair: rate - dur (r = 0.88)
  → Keeping 'dur', dropping 'rate'
Highly correlated pair: dttl - dur (r = 0.82)
  → Keeping 'dur', dropping 'dttl'
Highly correlated pair: dttl - state (r = 0.87)
  → Keeping 'state', dropping 'dttl'
Highly correlated pair: dttl - dpkts (r = 0.83)
  →

#### KDD

In [26]:
spearman_corr_kdd = df_kdd99_training.corr(method='spearman').abs()

# Create a set to store columns to keep and drop
cols_to_keep = set(spearman_corr_kdd.columns)
cols_to_drop = set()

# Iterate through the correlation matrix
for i in range(len(spearman_corr_kdd.columns)):
    for j in range(i):
        if spearman_corr_kdd.iloc[i, j] > 0.8:  # Check if correlation > 0.5
            col_i = spearman_corr_kdd.columns[i]
            col_j = spearman_corr_kdd.columns[j]
            
            # Print the correlated pair
            original_corr = spearman_corr_kdd.loc[col_i, col_j]
            print(f"Highly correlated pair: {col_i} - {col_j} (r = {original_corr:.2f})")
            
            # Keep the first one and drop the second (you can modify this logic)
            if col_j in cols_to_keep:
                cols_to_drop.add(col_i)
                print(f"  → Keeping '{col_j}', dropping '{col_i}'")

# Convert to lists
cols_to_keep = list(cols_to_keep - cols_to_drop)
cols_to_drop = list(cols_to_drop)

# Create filtered dataframe
df_kdd_filtered_spearman = spearman_corr_kdd[cols_to_keep]

# Print summary
print("\nVariables kept:", cols_to_keep)
print("Variables dropped:", cols_to_drop)
print("\nOriginal shape:", spearman_corr_kdd.shape)
print("Filtered shape:", df_kdd_filtered_spearman.shape)

Highly correlated pair: src_bytes - flag (r = 0.85)
  → Keeping 'flag', dropping 'src_bytes'
Highly correlated pair: logged_in - src_bytes (r = 0.86)
  → Keeping 'src_bytes', dropping 'logged_in'
Highly correlated pair: logged_in - dst_bytes (r = 0.86)
  → Keeping 'dst_bytes', dropping 'logged_in'
Highly correlated pair: srv_serror_rate - serror_rate (r = 0.99)
  → Keeping 'serror_rate', dropping 'srv_serror_rate'
Highly correlated pair: srv_rerror_rate - rerror_rate (r = 0.97)
  → Keeping 'rerror_rate', dropping 'srv_rerror_rate'
Highly correlated pair: same_srv_rate - flag (r = 0.81)
  → Keeping 'flag', dropping 'same_srv_rate'
Highly correlated pair: same_srv_rate - count (r = 0.81)
  → Keeping 'count', dropping 'same_srv_rate'
Highly correlated pair: same_srv_rate - serror_rate (r = 0.80)
  → Keeping 'serror_rate', dropping 'same_srv_rate'
Highly correlated pair: diff_srv_rate - same_srv_rate (r = 0.93)
  → Keeping 'same_srv_rate', dropping 'diff_srv_rate'
Highly correlated pair: d

### Remove Useless Columns from Correlation

In [27]:
# UNSW
new_feature_to_drop_unsw = ['sload', 'dmean', 'dbytes', 'dtcpb', 'ct_dst_src_ltm', 'synack', 'dttl', 'djit', 'tcprtt', 'dwin', 'sbytes', 'ct_state_ttl', 'dinpkt', 'ct_ftp_cmd', 'swin', 'ct_dst_sport_ltm', 'dpkts', 'rate', 'dloss', 'sinpkt', 'dload', 'ct_flw_http_mthd', 'sloss', 'ackdat', 'spkts', 'stcpb', 'sjit', 'ct_srv_dst']
df_unsw_training = df_unsw_training.drop(columns=new_feature_to_drop_unsw)
df_unsw_testing = df_unsw_testing.drop(columns=new_feature_to_drop_unsw)
df_autoencoder_unsw_train = df_autoencoder_unsw_train.drop(columns=new_feature_to_drop_unsw)

In [28]:
# KDD  
new_feature_to_drop_kdd = ['dst_host_srv_rerror_rate', 'same_srv_rate', 'src_bytes', 'dst_host_srv_serror_rate', 'srv_rerror_rate', 'dst_host_srv_diff_host_rate', 'dst_host_same_srv_rate', 'dst_host_serror_rate', 'dst_host_rerror_rate', 'logged_in', 'diff_srv_rate', 'dst_host_diff_srv_rate', 'srv_serror_rate']
df_kdd99_training = df_kdd99_training.drop(columns=new_feature_to_drop_kdd)
df_kdd99_testing = df_kdd99_testing.drop(columns=new_feature_to_drop_kdd)
df_autoencoder_kdd99_train = df_autoencoder_kdd99_train.drop(columns=new_feature_to_drop_kdd)

### Split Data

In [29]:
#### KDD99

df_kdd99_train, df_kdd99_validation, df_kdd99_train_label, df_kdd99_validation_label = train_test_split(
    df_kdd99_training, df_kdd99_training_label, test_size=0.2, random_state=42, stratify=df_kdd99_training_label
)

# testing
df_kdd99_testing_label = df_kdd99_testing['label']
df_kdd99_testing = df_kdd99_testing.drop(columns=columns_to_drop_kdd)

In [30]:
#### UNSW

df_unsw_train, df_unsw_validation, df_unsw_train_label, df_unsw_validation_label = train_test_split(
    df_unsw_training, df_unsw_training_label, test_size=0.2, random_state=42, stratify=df_unsw_training_label
)

# testing
df_unsw_testing_label = df_unsw_testing['label']
df_unsw_testing = df_unsw_testing.drop(columns=columns_to_drop_unsw)

### Change labels to attack or not attack

In [31]:
df_kdd99_train_label = df_kdd99_train_label.apply(lambda x: 0 if x == 'normal' else 1)
df_kdd99_validation_label = df_kdd99_validation_label.apply(lambda x: 0 if x == 'normal' else 1)
df_kdd99_testing_label = df_kdd99_testing_label.apply(lambda x: 0 if x == 'normal.' else 1)

### See Feature Importance to Label

In [32]:
correlations = df_unsw_train.corrwith(df_unsw_train_label).abs()
important_features = correlations.sort_values(ascending=False)
print(important_features)


sttl                 0.691462
state                0.497449
ct_src_dport_ltm     0.305160
ct_src_ltm           0.237933
ct_dst_ltm           0.229537
ct_srv_src           0.229100
is_sm_ips_ports      0.184356
service              0.036188
dur                  0.036169
response_body_len    0.020914
proto                0.019001
smean                0.012492
trans_depth          0.011220
is_ftp_login         0.010597
dtype: float64


In [33]:
correlations_kdd = df_kdd99_train.corrwith(df_kdd99_train_label).abs()
important_features_kdd = correlations_kdd.sort_values(ascending=False)
print(important_features_kdd)


count                          0.805636
dst_host_srv_count             0.790906
serror_rate                    0.788581
flag                           0.719519
service                        0.610186
dst_host_count                 0.519942
srv_diff_host_rate             0.269555
rerror_rate                    0.216971
protocol_type                  0.195131
dst_host_same_src_port_rate    0.155192
wrong_fragment                 0.104029
duration                       0.057312
hot                            0.043497
srv_count                      0.042521
num_access_files               0.038271
dst_bytes                      0.018836
num_failed_logins              0.013298
land                           0.012958
num_file_creations             0.009636
root_shell                     0.008632
is_guest_login                 0.008622
num_root                       0.007385
num_shells                     0.006040
su_attempted                   0.005192
num_compromised                0.002019


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


### Remove Useless Columns from Feature Importance

In [34]:
# KDD  
new_feature_to_drop_kdd = ["num_outbound_cmds", "is_host_login"]
df_kdd99_train = df_kdd99_train.drop(columns=new_feature_to_drop_kdd)
df_kdd99_validation = df_kdd99_validation.drop(columns=new_feature_to_drop_kdd)
df_autoencoder_kdd99_train = df_autoencoder_kdd99_train.drop(columns=new_feature_to_drop_kdd)
df_kdd99_testing = df_kdd99_testing.drop(columns=new_feature_to_drop_kdd)


### Check distribution

In [35]:
# Training dataset anomaly percentage
anomalous_kdd99_train = (df_kdd99_train_label.sum() / len(df_kdd99_train_label)) * 100

# Validation dataset anomaly percentage
anomalous_kdd99_validation = (df_kdd99_validation_label.sum() / len(df_kdd99_validation_label)) * 100

# Testing dataset anomaly percentage
anomalous_kdd99_testing = (df_kdd99_testing_label.sum() / len(df_kdd99_testing_label)) * 100

print(f"KDD99 Training Anomaly Percentage: {anomalous_kdd99_train:.2f}%")
print(f"KDD99 Validation Anomaly Percentage: {anomalous_kdd99_validation:.2f}%")
print(f"KDD99 Testing Anomaly Percentage: {anomalous_kdd99_testing:.2f}%")


KDD99 Training Anomaly Percentage: 39.67%
KDD99 Validation Anomaly Percentage: 39.67%
KDD99 Testing Anomaly Percentage: 42.17%


In [36]:
# Training dataset anomaly percentage
anomalous_unsw_train = (df_unsw_train_label.sum() / len(df_unsw_train_label)) * 100

# Validation dataset anomaly percentage
anomalous_unsw_validation = (df_unsw_validation_label.sum() / len(df_unsw_validation_label)) * 100

# Testing dataset anomaly percentage
anomalous_unsw_testing = (df_unsw_testing_label.sum() / len(df_unsw_testing_label)) * 100

print(f"UNSW Training Anomaly Percentage: {anomalous_unsw_train:.2f}%")
print(f"UNSW Validation Anomaly Percentage: {anomalous_unsw_validation:.2f}%")
print(f"UNSW Testing Anomaly Percentage: {anomalous_unsw_testing:.2f}%")


UNSW Training Anomaly Percentage: 68.06%
UNSW Validation Anomaly Percentage: 68.06%
UNSW Testing Anomaly Percentage: 55.06%


### Normalize Data

In [37]:
scaler_kdd = RobustScaler()
scaler_unsw = RobustScaler()

# kdd
df_kdd99_train = pd.DataFrame(scaler_kdd.fit_transform(df_kdd99_train),columns=df_kdd99_train.columns)
df_kdd99_testing = pd.DataFrame(scaler_kdd.transform(df_kdd99_testing), columns=df_kdd99_testing.columns)
df_autoencoder_kdd99_train = pd.DataFrame(scaler_kdd.transform(df_autoencoder_kdd99_train),columns=df_autoencoder_kdd99_train.columns)
df_kdd99_validation = pd.DataFrame(scaler_kdd.transform(df_kdd99_validation),columns=df_kdd99_validation.columns)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler_kdd, f)

# unsw
df_unsw_train= pd.DataFrame(scaler_unsw.fit_transform(df_unsw_train),columns=df_unsw_train.columns)
df_unsw_testing = pd.DataFrame(scaler_unsw.transform(df_unsw_testing), columns=df_unsw_testing.columns)
df_autoencoder_unsw_train = pd.DataFrame(scaler_unsw.transform(df_autoencoder_unsw_train),columns=df_autoencoder_unsw_train.columns)
df_unsw_validation = pd.DataFrame(scaler_unsw.transform(df_unsw_validation),columns=df_unsw_validation.columns)


### Final Check for Dataset

#### KDD99

In [38]:
# kdd
df_kdd99_train_label.unique()


array([0, 1], dtype=int64)

In [39]:
df_kdd99_testing_label.unique()


array([1, 0], dtype=int64)

In [40]:
df_kdd99_testing.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97325 entries, 0 to 97324
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     97325 non-null  float64
 1   protocol_type                97325 non-null  float64
 2   service                      97325 non-null  float64
 3   flag                         97325 non-null  float64
 4   dst_bytes                    97325 non-null  float64
 5   land                         97325 non-null  float64
 6   wrong_fragment               97325 non-null  float64
 7   urgent                       97325 non-null  float64
 8   hot                          97325 non-null  float64
 9   num_failed_logins            97325 non-null  float64
 10  num_compromised              97325 non-null  float64
 11  root_shell                   97325 non-null  float64
 12  su_attempted                 97325 non-null  float64
 13  num_root        

In [41]:
df_autoencoder_kdd99_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87831 entries, 0 to 87830
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     87831 non-null  float64
 1   protocol_type                87831 non-null  float64
 2   service                      87831 non-null  float64
 3   flag                         87831 non-null  float64
 4   dst_bytes                    87831 non-null  float64
 5   land                         87831 non-null  float64
 6   wrong_fragment               87831 non-null  float64
 7   urgent                       87831 non-null  float64
 8   hot                          87831 non-null  float64
 9   num_failed_logins            87831 non-null  float64
 10  num_compromised              87831 non-null  float64
 11  root_shell                   87831 non-null  float64
 12  su_attempted                 87831 non-null  float64
 13  num_root        

In [42]:
df_kdd99_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116468 entries, 0 to 116467
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     116468 non-null  float64
 1   protocol_type                116468 non-null  float64
 2   service                      116468 non-null  float64
 3   flag                         116468 non-null  float64
 4   dst_bytes                    116468 non-null  float64
 5   land                         116468 non-null  float64
 6   wrong_fragment               116468 non-null  float64
 7   urgent                       116468 non-null  float64
 8   hot                          116468 non-null  float64
 9   num_failed_logins            116468 non-null  float64
 10  num_compromised              116468 non-null  float64
 11  root_shell                   116468 non-null  float64
 12  su_attempted                 116468 non-null  float64
 13 

In [43]:
df_kdd99_validation.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29117 entries, 0 to 29116
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     29117 non-null  float64
 1   protocol_type                29117 non-null  float64
 2   service                      29117 non-null  float64
 3   flag                         29117 non-null  float64
 4   dst_bytes                    29117 non-null  float64
 5   land                         29117 non-null  float64
 6   wrong_fragment               29117 non-null  float64
 7   urgent                       29117 non-null  float64
 8   hot                          29117 non-null  float64
 9   num_failed_logins            29117 non-null  float64
 10  num_compromised              29117 non-null  float64
 11  root_shell                   29117 non-null  float64
 12  su_attempted                 29117 non-null  float64
 13  num_root        

#### UNSW

In [44]:
df_unsw_testing.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                82332 non-null  float64
 1   proto              82332 non-null  float64
 2   service            82332 non-null  float64
 3   state              82332 non-null  float64
 4   sttl               82332 non-null  float64
 5   smean              82332 non-null  float64
 6   trans_depth        82332 non-null  float64
 7   response_body_len  82332 non-null  float64
 8   ct_srv_src         82332 non-null  float64
 9   ct_dst_ltm         82332 non-null  float64
 10  ct_src_dport_ltm   82332 non-null  float64
 11  is_ftp_login       82332 non-null  float64
 12  ct_src_ltm         82332 non-null  float64
 13  is_sm_ips_ports    82332 non-null  float64
dtypes: float64(14)
memory usage: 8.8 MB


In [45]:
df_autoencoder_unsw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                56000 non-null  float64
 1   proto              56000 non-null  float64
 2   service            56000 non-null  float64
 3   state              56000 non-null  float64
 4   sttl               56000 non-null  float64
 5   smean              56000 non-null  float64
 6   trans_depth        56000 non-null  float64
 7   response_body_len  56000 non-null  float64
 8   ct_srv_src         56000 non-null  float64
 9   ct_dst_ltm         56000 non-null  float64
 10  ct_src_dport_ltm   56000 non-null  float64
 11  is_ftp_login       56000 non-null  float64
 12  ct_src_ltm         56000 non-null  float64
 13  is_sm_ips_ports    56000 non-null  float64
dtypes: float64(14)
memory usage: 6.0 MB


In [46]:
df_unsw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140272 entries, 0 to 140271
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   dur                140272 non-null  float64
 1   proto              140272 non-null  float64
 2   service            140272 non-null  float64
 3   state              140272 non-null  float64
 4   sttl               140272 non-null  float64
 5   smean              140272 non-null  float64
 6   trans_depth        140272 non-null  float64
 7   response_body_len  140272 non-null  float64
 8   ct_srv_src         140272 non-null  float64
 9   ct_dst_ltm         140272 non-null  float64
 10  ct_src_dport_ltm   140272 non-null  float64
 11  is_ftp_login       140272 non-null  float64
 12  ct_src_ltm         140272 non-null  float64
 13  is_sm_ips_ports    140272 non-null  float64
dtypes: float64(14)
memory usage: 15.0 MB


In [47]:
df_unsw_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35069 entries, 0 to 35068
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                35069 non-null  float64
 1   proto              35069 non-null  float64
 2   service            35069 non-null  float64
 3   state              35069 non-null  float64
 4   sttl               35069 non-null  float64
 5   smean              35069 non-null  float64
 6   trans_depth        35069 non-null  float64
 7   response_body_len  35069 non-null  float64
 8   ct_srv_src         35069 non-null  float64
 9   ct_dst_ltm         35069 non-null  float64
 10  ct_src_dport_ltm   35069 non-null  float64
 11  is_ftp_login       35069 non-null  float64
 12  ct_src_ltm         35069 non-null  float64
 13  is_sm_ips_ports    35069 non-null  float64
dtypes: float64(14)
memory usage: 3.7 MB


In [48]:
df_unsw_train_label.unique()

array([1, 0], dtype=int64)

In [49]:
df_kdd99_testing_label.unique()

array([1, 0], dtype=int64)

## **FINAL USED MODEL GENERATION**

### +++++++++++++++++++++++++++++++++++++++++++++++ **DBSCAN** +++++++++++++++++++++++++++++++++++++++++++++++

#### ************Architecture************

In [50]:
def create_dbscan(eps, min_samples):
    """Create a DBSCAN model with specified parameters."""
    return DBSCAN(
        eps=float(eps),
        min_samples=int(min_samples),
        n_jobs=-1
    )

In [51]:
def objective_function(solution, X_train, X_val, y_val):
    eps, min_samples = solution
    dbscan = create_dbscan(eps=eps, min_samples=int(min_samples))
    dbscan.fit(X_train)
    y_pred = (dbscan.fit_predict(X_val) == -1).astype(int)
    if len(np.unique(y_pred)) == 1:
        return 1.0
    return 1 - f1_score(y_val, y_pred)

In [60]:
def optimize_dbscan(X_train, X_val, y_val, optimizer_type, epochs=15, pop_size=10):
    problem_dict = {
        "bounds": FloatVar(lb=[0.1, 2], ub=[1.0, 20], name="delta"),
        "minmax": "min",
        "obj_func": lambda solution: objective_function(solution, X_train, X_val, y_val)
    }

    # Choose Optimizer
    if optimizer_type == "WOA":
        model = WOA.HI_WOA(epoch=epochs, pop_size=pop_size, feedback_max=4)
    elif optimizer_type == "DE":
        model = DE.JADE(epoch=epochs, pop_size=pop_size,
                        miu_f=0.5, miu_cr=0.5, pt=0.1, ap=0.1)
    
    # Run Optimization
    g_best = model.solve(problem_dict)
    return g_best.solution



In [53]:
def evaluate_model(model, X_train, X_test, y_test):
    model.fit(X_train)
    test_labels = model.fit_predict(X_test)
    y_pred = (test_labels == -1).astype(int)
    
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred),
        "conf": confusion_matrix(y_test, y_pred)

    }
    y_scores = (test_labels == -1).astype(float)
    metrics["auc"] = roc_auc_score(y_test, y_scores)
    
    return metrics


In [54]:
def main_pipeline_dbscan(dataset_name, optimizer_type):
    if dataset_name == "kdd99":
        X_train = df_kdd99_training
        X_val = df_kdd99_validation
        y_val = df_kdd99_validation_label
        X_test = df_kdd99_testing
        y_test = df_kdd99_testing_label
    elif dataset_name == "unsw":
        X_train = df_unsw_train
        X_val = df_unsw_validation
        y_val = df_unsw_validation_label
        X_test = df_unsw_testing
        y_test = df_unsw_testing_label
     
    if optimizer_type == "default":
        eps, min_samples = 0.5, 5
        print(f"\nUsing default DBSCAN parameters for {dataset_name}")
    else:
        best_params = optimize_dbscan(X_train, X_val, y_val, optimizer_type)
        eps, min_samples = best_params
        print(f"\nBest Parameters found using {optimizer_type}:")
        print(f"eps: {eps:.4f}, min_samples: {int(min_samples)}")
   
    final_model = create_dbscan(eps=eps, min_samples=int(min_samples))
    
    # Evaluate on test set using distance-based approach
    metrics = evaluate_model(final_model, X_train, X_test, y_test)

    filename = dataset_name + "_" + optimizer_type + "_" + "DBSCAN.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(final_model, f)
    
    # Print results
    print(f"\n=== Results for {dataset_name} with {optimizer_type} ===")
    print("\nBest Parameters:")
    print(f"eps: {eps:.4f}")
    print(f"min_samples: {int(min_samples)}")
    
    print("\nTest Set Metrics:")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    print(f"AUC: {metrics['auc']:.4f}")
    print(f"confusion_matrix: \n{ metrics['conf']}")
    

#### ************DBSCAN only************

In [55]:
main_pipeline_dbscan('unsw', 'default')


Using default DBSCAN parameters for unsw

=== Results for unsw with default ===

Best Parameters:
eps: 0.5000
min_samples: 5

Test Set Metrics:
Accuracy: 0.4615
Precision: 0.5799
Recall: 0.0796
F1 Score: 0.1400
AUC: 0.5045
confusion_matrix: 
[[34386  2614]
 [41723  3609]]


In [56]:
main_pipeline_dbscan('kdd99', 'default')


Using default DBSCAN parameters for kdd99

=== Results for kdd99 with default ===

Best Parameters:
eps: 0.5000
min_samples: 5

Test Set Metrics:
Accuracy: 0.5225
Precision: 0.0824
Recall: 0.0130
F1 Score: 0.0225
AUC: 0.4536
confusion_matrix: 
[[50320  5959]
 [40511   535]]


#### ************DBSCAN + Whale Optimization Algorithm************

In [57]:
main_pipeline_dbscan('unsw', 'WOA')


2025/04/28 01:51:50 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.
2025/04/28 01:54:45 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: 0.4345259232954546, Global best: 0.4345259232954546, Runtime: 59.67381 seconds
2025/04/28 01:55:32 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 2, Current best: 0.40394700815097195, Global best: 0.40394700815097195, Runtime: 46.30965 seconds
2025/04/28 01:56:11 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 3, Current best: 0.40394700815097195, Global best: 0.40394700815097195, Runtime: 39.30370 seconds
2025/04/28 01:56:50 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 4, Current best: 0.40394700815097195, Global best: 0.40394700815097195, Runtime: 39.39878 seconds
2025/04/28 01:57:24 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 5, Current best: 0.40394700815097195, Global best: 0.40394700815097195, Runtime: 33.891


Best Parameters found using WOA:
eps: 0.1000, min_samples: 9

=== Results for unsw with WOA ===

Best Parameters:
eps: 0.1000
min_samples: 9

Test Set Metrics:
Accuracy: 0.3244
Precision: 0.3786
Recall: 0.3538
F1 Score: 0.3658
AUC: 0.3211
confusion_matrix: 
[[10672 26328]
 [29293 16039]]


In [58]:
main_pipeline_dbscan('kdd99', 'WOA')

2025/04/28 02:03:50 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.
2025/04/28 02:04:23 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: 0.8565129305207961, Global best: 0.8565129305207961, Runtime: 16.73359 seconds
2025/04/28 02:04:40 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 2, Current best: 0.8565129305207961, Global best: 0.8565129305207961, Runtime: 16.58327 seconds
2025/04/28 02:04:57 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 3, Current best: 0.8521479328165374, Global best: 0.8521479328165374, Runtime: 16.81280 seconds
2025/04/28 02:05:13 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 4, Current best: 0.8521479328165374, Global best: 0.8521479328165374, Runtime: 16.67567 seconds
2025/04/28 02:05:30 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 5, Current best: 0.8515628048542553, Global best: 0.8515628048542553, Runtime: 16.21288 secon


Best Parameters found using WOA:
eps: 0.1296, min_samples: 9

=== Results for kdd99 with WOA ===

Best Parameters:
eps: 0.1296
min_samples: 9

Test Set Metrics:
Accuracy: 0.3145
Precision: 0.1115
Recall: 0.0897
F1 Score: 0.0994
AUC: 0.2841
confusion_matrix: 
[[26926 29353]
 [37363  3683]]


#### ************DBSCAN + Binary Differential Evolution Algorithm************

In [61]:
main_pipeline_dbscan('kdd99', 'DE')

2025/04/28 12:59:46 PM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.
2025/04/28 01:00:52 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: 0.8373089953674895, Global best: 0.8373089953674895, Runtime: 32.94982 seconds
2025/04/28 01:01:26 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 2, Current best: 0.8342985623765075, Global best: 0.8342985623765075, Runtime: 33.79166 seconds
2025/04/28 01:01:58 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 3, Current best: 0.8342985623765075, Global best: 0.8342985623765075, Runtime: 32.48831 seconds
2025/04/28 01:02:31 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 4, Current best: 0.8342985623765075, Global best: 0.8342985623765075, Runtime: 33.14799 seconds
2025/04/28 01:03:04 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 5, Current best: 0.8342985623765075, Global best: 0.8342985623765075,


Best Parameters found using DE:
eps: 0.1000, min_samples: 20

=== Results for kdd99 with DE ===

Best Parameters:
eps: 0.1000
min_samples: 20

Test Set Metrics:
Accuracy: 0.2011
Precision: 0.1140
Recall: 0.1321
F1 Score: 0.1224
AUC: 0.1917
confusion_matrix: 
[[14148 42131]
 [35624  5422]]


In [62]:
main_pipeline_dbscan('unsw', 'DE')

2025/04/28 01:08:53 PM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.
2025/04/28 01:12:53 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: 0.3657642413987592, Global best: 0.3657642413987592, Runtime: 126.79649 seconds
2025/04/28 01:14:41 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 2, Current best: 0.3657642413987592, Global best: 0.3657642413987592, Runtime: 108.45176 seconds
2025/04/28 01:15:35 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 3, Current best: 0.35048053106113153, Global best: 0.35048053106113153, Runtime: 53.72580 seconds
2025/04/28 01:16:42 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 4, Current best: 0.35048053106113153, Global best: 0.35048053106113153, Runtime: 66.91991 seconds
2025/04/28 01:18:25 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 5, Current best: 0.3325303545410393, Global best: 0.33253035454


Best Parameters found using DE:
eps: 0.1000, min_samples: 20

=== Results for unsw with DE ===

Best Parameters:
eps: 0.1000
min_samples: 20

Test Set Metrics:
Accuracy: 0.3862
Precision: 0.4550
Recall: 0.5792
F1 Score: 0.5096
AUC: 0.3645
confusion_matrix: 
[[ 5542 31458]
 [19074 26258]]


### +++++++++++++++++++++++++++++++++++++++++++++++ **Isolation Forest** +++++++++++++++++++++++++++++++++++++++++++++++

#### ************Architecture************

In [50]:
def create_isolation_forest(n_estimators, max_samples, contamination, max_features, bootstrap, random_state):
    return IsolationForest(
        n_estimators=n_estimators,
        max_samples=max_samples,
        contamination=contamination,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=random_state,
    )

In [51]:
def objective_function(solution, X_train, X_val, y_val):

    n_estimators, max_samples, contamination, max_features, bootstrap, random_state = solution
    bootstrap = bool(round(bootstrap))
    random_state=int(random_state)

    y_val_binary = np.where(y_val > 0, 1, 0)

    model = create_isolation_forest(
        n_estimators=int(n_estimators),
        max_samples=int(max_samples),
        contamination=contamination,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=random_state,
    )
    
    model.fit(X_train)
    
    train_scores = -model.decision_function(X_train)
    threshold = np.percentile(train_scores, 100 * (1 - contamination))
        
    # Then evaluate on validation
    val_scores = -model.decision_function(X_val)
    y_pred = (val_scores > threshold).astype(int)
    
    return 1 - f1_score(y_val_binary, y_pred)

In [52]:

def optimize_isolation_forest(X_train, X_val, y_val, optimizer_type, epochs=15, pop_size=10):
   
    problem_dict = {
        "bounds": FloatVar(
            lb=[
                50,     # n_estimators (50-500)
                100,    # max_samples (100-1000)
                0.01,   # contamination (0.01-0.5)
                0.1,    # max_features (0.1-1.0)
                0,      # bootstrap (0=False, 1=True)
                0       # random_state (0-100)
            ],
            ub=[
                500,1000,0.5,1.0,1,100
            ],
            name="delta",
        ),
        "minmax": "min",
        "obj_func": lambda solution: objective_function(solution, X_train, X_val, y_val),
    }
    
    # Choose Optimizer
    if optimizer_type == "WOA":
        model = WOA.HI_WOA(epoch=epochs, pop_size=pop_size, feedback_max=4)
    elif optimizer_type == "DE":
        model = DE.JADE(epoch=epochs, pop_size=pop_size, miu_f=0.5, miu_cr=0.5, pt=0.1, ap=0.1)
    
    # Run Optimization
    g_best = model.solve(problem_dict)
    return g_best.solution


In [53]:

def evaluate_model(model, X_train, X_test, y_test):
    
    train_scores = -model.decision_function(X_train)
    threshold = np.percentile(train_scores, 100 * (1 - model.contamination))
    
    # Evaluate on test data
    test_scores = -model.decision_function(X_test)
    y_pred = (test_scores > threshold).astype(int)
    
    # Compute Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, test_scores)
    conf = confusion_matrix(y_test, y_pred)

    return accuracy, precision, recall, f1, auc, conf, threshold


In [54]:
def main_pipeline_isolation_forest(dataset_name, optimizer_type):

    if dataset_name == "kdd99":
        X_train = df_kdd99_train
        X_val = df_kdd99_validation
        y_val = df_kdd99_validation_label
        X_test = df_kdd99_testing
        y_test = df_kdd99_testing_label
    elif dataset_name == "unsw":
        X_train = df_unsw_train
        X_val = df_unsw_validation
        y_val = df_unsw_validation_label
        X_test = df_unsw_testing
        y_test = df_unsw_testing_label
  
    if optimizer_type == "default":
        n_estimators = 100
        max_samples = 'auto' 
        if dataset_name == "kdd99":
            contamination = 0.4
        else:
            contamination = 0.5
        max_features = 1.0 
        bootstrap = False
        random_state = None
    else:
        best_params = optimize_isolation_forest(X_train, X_val, y_val, optimizer_type)
            
        # Extract parameters
        (n_estimators, max_samples, contamination, 
        max_features, bootstrap, random_state) = best_params
        random_state=int(random_state)
        max_samples = int(max_samples)
        # Convert binary bootstrap flag to boolean
        bootstrap = bool(round(bootstrap))
    
    # Create final model with best parameters
    model = create_isolation_forest(
        n_estimators=int(n_estimators),
        max_samples=max_samples,
        contamination=contamination,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=random_state
    )
    
    # Train final model
    model.fit(X_train)
    # Evaluate on test set
    accuracy, precision, recall, f1, auc, conf, threshold = evaluate_model(model, X_train, X_test, y_test)

    filename = dataset_name + "_" + optimizer_type + "_" + "isolation_forest.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(model, f)

    with open('threshold.pkl', 'wb') as d:
        pickle.dump(threshold, d)
        
    # Print results
    print(f"\n=== For Dataset {dataset_name}, {optimizer_type} optimizer ===")
    
    print("\n=== Best Parameters ===")
    print(f"n_estimators: {int(n_estimators)}")
    print(f"max_samples: {max_samples}")
    print(f"contamination: {contamination:.4f}")
    print(f"max_features: {max_features:.4f}")
    print(f"bootstrap: {bootstrap}")
    print(f"random_state: {random_state}")
    print(f"Threshold: {threshold}")
    
    print("\n=== Evaluation Metrics ===")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"confusion matrix: \n{conf}")


#### ************Isolation Forest************

In [68]:
main_pipeline_isolation_forest('unsw', 'default')



=== For Dataset unsw, default optimizer ===

=== Best Parameters ===
n_estimators: 100
max_samples: auto
contamination: 0.5000
max_features: 1.0000
bootstrap: False
random_state: None

=== Evaluation Metrics ===
Accuracy: 0.4731
Precision: 0.5251
Recall: 0.4504
F1 Score: 0.4849
AUC: 0.5179
confusion matrix: 
[[18535 18465]
 [24915 20417]]


In [69]:
main_pipeline_isolation_forest('kdd99', 'default')


=== For Dataset kdd99, default optimizer ===

=== Best Parameters ===
n_estimators: 100
max_samples: auto
contamination: 0.4000
max_features: 1.0000
bootstrap: False
random_state: None

=== Evaluation Metrics ===
Accuracy: 0.5223
Precision: 0.4279
Recall: 0.3940
F1 Score: 0.4103
AUC: 0.6023
confusion matrix: 
[[34661 21618]
 [24874 16172]]


#### ************Isolation Forest + Whale Optimization Algorithm************

In [95]:
main_pipeline_isolation_forest('kdd99', 'WOA')

2025/04/28 02:02:33 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.
2025/04/28 02:03:16 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: 0.4646522452405478, Global best: 0.4646522452405478, Runtime: 20.62723 seconds
2025/04/28 02:03:37 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 2, Current best: 0.4582490940301355, Global best: 0.4582490940301355, Runtime: 21.51823 seconds
2025/04/28 02:04:00 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 3, Current best: 0.43706866969153924, Global best: 0.43706866969153924, Runtime: 22.33644 seconds
2025/04/28 02:04:26 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 4, Current best: 0.43706866969153924, Global best: 0.43706866969153924, Runtime: 26.22488 seconds
2025/04/28 02:04:53 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 5, Current best: 0.43706866969153924, Global best: 0.43706866969153924, Runtime: 26.93491


=== For Dataset kdd99, WOA optimizer ===

=== Best Parameters ===
n_estimators: 473
max_samples: 1000
contamination: 0.5000
max_features: 1.0000
bootstrap: True
random_state: 72

=== Evaluation Metrics ===
Accuracy: 0.5924
Precision: 0.5142
Recall: 0.6046
F1 Score: 0.5558
AUC: 0.6479
confusion matrix: 
[[32833 23446]
 [16228 24818]]


In [96]:
main_pipeline_isolation_forest('unsw', 'WOA')

2025/04/28 02:10:38 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.
2025/04/28 02:11:35 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: 0.46635786832645776, Global best: 0.46635786832645776, Runtime: 27.66222 seconds
2025/04/28 02:12:08 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 2, Current best: 0.4609548661447762, Global best: 0.4609548661447762, Runtime: 33.09820 seconds
2025/04/28 02:12:43 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 3, Current best: 0.4609548661447762, Global best: 0.4609548661447762, Runtime: 35.00529 seconds
2025/04/28 02:13:21 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 4, Current best: 0.4609548661447762, Global best: 0.4609548661447762, Runtime: 38.34044 seconds
2025/04/28 02:14:03 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 5, Current best: 0.4609548661447762, Global best: 0.4609548661447762, Runtime: 42.04495 sec


=== For Dataset unsw, WOA optimizer ===

=== Best Parameters ===
n_estimators: 500
max_samples: 959
contamination: 0.5000
max_features: 0.9927
bootstrap: True
random_state: 98

=== Evaluation Metrics ===
Accuracy: 0.4669
Precision: 0.5178
Recall: 0.4621
F1 Score: 0.4883
AUC: 0.5159
confusion matrix: 
[[17493 19507]
 [24386 20946]]


#### ************Isolation Forest + Binary Differential Evolution Algorithm************

In [55]:
main_pipeline_isolation_forest('kdd99', 'DE')

2025/04/30 02:25:07 PM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.
2025/04/30 02:28:34 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: 0.6259320826407029, Global best: 0.6259320826407029, Runtime: 115.78403 seconds
2025/04/30 02:30:44 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 2, Current best: 0.512068438741216, Global best: 0.512068438741216, Runtime: 129.92183 seconds
2025/04/30 02:33:09 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 3, Current best: 0.4935396577352593, Global best: 0.4935396577352593, Runtime: 145.07720 seconds
2025/04/30 02:35:45 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 4, Current best: 0.4686369349125947, Global best: 0.4686369349125947, Runtime: 155.78726 seconds
2025/04/30 02:38:20 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 5, Current best: 0.45714394894722765, Global best: 0.45714394894722


=== For Dataset kdd99, DE optimizer ===

=== Best Parameters ===
n_estimators: 500
max_samples: 1000
contamination: 0.5000
max_features: 0.9629
bootstrap: True
random_state: 0
Threshold: -2.7755575615628914e-17

=== Evaluation Metrics ===
Accuracy: 0.5740
Precision: 0.4957
Recall: 0.5810
F1 Score: 0.5350
AUC: 0.6376
confusion matrix: 
[[32015 24264]
 [17198 23848]]


In [98]:
main_pipeline_isolation_forest('unsw', 'DE')

2025/04/28 02:23:34 PM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.
2025/04/28 02:24:21 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: 0.5078013830291834, Global best: 0.5078013830291834, Runtime: 25.97592 seconds
2025/04/28 02:24:48 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 2, Current best: 0.47070453778270727, Global best: 0.47070453778270727, Runtime: 27.31626 seconds
2025/04/28 02:25:13 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 3, Current best: 0.46011176413912713, Global best: 0.46011176413912713, Runtime: 25.11117 seconds
2025/04/28 02:25:32 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 4, Current best: 0.4282473331080755, Global best: 0.4282473331080755, Runtime: 19.33398 seconds
2025/04/28 02:25:42 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 5, Current best: 0.4282473331080755, Global best: 0.4282473331080


=== For Dataset unsw, DE optimizer ===

=== Best Parameters ===
n_estimators: 50
max_samples: 841
contamination: 0.5000
max_features: 0.5316
bootstrap: False
random_state: 71

=== Evaluation Metrics ===
Accuracy: 0.5226
Precision: 0.5772
Recall: 0.4967
F1 Score: 0.5340
AUC: 0.5574
confusion matrix: 
[[20509 16491]
 [22815 22517]]


### +++++++++++++++++++++++++++++++++++++++++++++++ **Autoencoder** +++++++++++++++++++++++++++++++++++++++++++++++

#### ************Architecture************

In [99]:
def create_autoencoder(
    input_dim,latent_dim,neurons_per_layer,activation_function, output_activation, loss_function, optimizer, learning_rate, dropout_rate):
    # Input Layer
    input_layer = Input(shape=(input_dim,))
    
    # Encoder
    x = Dense(neurons_per_layer, activation=activation_function)(input_layer)
    x = Dropout(dropout_rate)(x)
    x = Dense(neurons_per_layer, activation=activation_function)(x)  # 2nd hidden layer
    x = Dropout(dropout_rate)(x)
    encoded = Dense(latent_dim, activation=activation_function)(x)  # Latent space
    
    # Decoder
    x = Dense(neurons_per_layer, activation=activation_function)(encoded)
    x = Dropout(dropout_rate)(x)
    x = Dense(neurons_per_layer, activation=activation_function)(x)  # 2nd hidden layer
    x = Dropout(dropout_rate)(x)
    decoded = Dense(input_dim, activation=output_activation)(x)  # Output layer
    
    # Autoencoder Model
    autoencoder = Model(input_layer, decoded)

    # Optimizer Selection
    if optimizer == "Adam":
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == "SGD":
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == "Nadam":
        opt = Nadam(learning_rate=learning_rate)
    
    # Compile Model
    autoencoder.compile(optimizer=opt, loss=loss_function)
    
    return autoencoder

In [100]:
def objective_function(solution, X_train, X_val, y_val, input_dim):
    (latent_dim, neurons_per_layer, learning_rate, dropout_rate, batch_size, epochs,
     activation_idx, output_activation_idx, loss_idx, optimizer_idx, threshold_percentile) = solution

    # Map indices to actual values
    activation_functions = ["relu", "tanh", "sigmoid"]
    output_activations = ["sigmoid", "tanh", "linear", "softmax", "relu"]
    loss_functions = ["mean_squared_error", "mean_absolute_error",
                      "binary_crossentropy", "categorical_crossentropy"]
    optimizers = ["Adam", "SGD", "Nadam"]

    activation_function = activation_functions[int(activation_idx)]
    output_activation = output_activations[int(output_activation_idx)]
    loss_function = loss_functions[int(loss_idx)]
    optimizer = optimizers[int(optimizer_idx)]

    # Create Autoencoder
    autoencoder = create_autoencoder(
        input_dim=input_dim,
        latent_dim=int(latent_dim),
        neurons_per_layer=int(neurons_per_layer),
        activation_function=activation_function,
        output_activation=output_activation,
        loss_function=loss_function,
        optimizer=optimizer,
        learning_rate=learning_rate,
        dropout_rate=dropout_rate,
    )

    # Train Autoencoder
    autoencoder.fit(
        X_train,
        X_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        verbose=0,
    )

    # Compute reconstruction error on training data for threshold calculation
    reconstructed_train = autoencoder.predict(X_train)
    mse_train = np.mean(np.square(X_train - reconstructed_train), axis=1)
    
    # Compute reconstruction error on validation data (fitness value)
    reconstructed_val = autoencoder.predict(X_val)
    mse_val = np.mean(np.square(X_val - reconstructed_val), axis=1)
    
    # Use optimized threshold percentile
    threshold = np.percentile(mse_train, threshold_percentile)
    
    # Calculate validation accuracy (fitness value to maximize)
    y_val_pred = (mse_val > threshold).astype(int)

    return 1 - f1_score(y_val, y_val_pred)  # Minimize (1 - F1)

In [101]:
def optimize_autoencoder(X_train, X_val, y_val, input_dim, optimizer_type, epochs=20, pop_size=10):
    # Define the search space
    problem_dict = {
        "bounds": FloatVar(
            lb=[
                16,    # latent_dim (16-64)
                16,    # neurons_per_layer (16-128)
                0.001, # learning_rate (0.001-0.01)
                0.1,   # dropout_rate (0.1-0.5)
                16,    # batch_size (16-256)
                20,    # epochs (20-100)
                0,     # activation_function index (0: relu, 1: tanh, 2: sigmoid)
                0,     # output_activation index (0: sigmoid, 1: tanh, 2: linear, 3: softmax, 4: relu)
                0,     # loss_function index (0: mse, 1: mae, 2: binary_crossentropy, 3: categorical_crossentropy)
                0,     # optimizer index (0: Adam, 1: SGD, 2: Nadam)
                95,    # threshold_percentile (95-99.9)
            ],
            ub=[
                64,128,0.01,0.5,256,100,2,4,3,2,99.9],
            name="delta",
        ),
        "minmax": "min", 
        "obj_func": lambda solution: objective_function(solution, X_train, X_val, y_val, input_dim),
    }
    
    # Choose Optimizer
    if optimizer_type == "WOA":
        model = WOA.HI_WOA(epoch=epochs, pop_size=pop_size, feedback_max=4)
    elif optimizer_type == "DE":
        model = DE.JADE(epoch=epochs, pop_size=pop_size, miu_f=0.5, miu_cr=0.5, pt=0.1, ap=0.1)

    # Run Optimization
    g_best = model.solve(problem_dict)
    return g_best.solution

In [102]:
def evaluate_model(autoencoder, X_train, X_test, y_test, threshold_percentile):
    reconstructed_train = autoencoder.predict(X_train)
    mse_train = np.mean(np.square(X_train - reconstructed_train), axis=1)
    
    # Calculate reconstruction error for test data
    reconstructed_test = autoencoder.predict(X_test)
    mse_test = np.mean(np.square(X_test - reconstructed_test), axis=1)
    
    threshold = np.percentile(mse_train, threshold_percentile)
    
    # Predict anomalies
    y_pred = (mse_test > threshold).astype(int)
    
    # Compute Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, mse_test)
    conf = confusion_matrix(y_test, y_pred)

    return accuracy, precision, recall, f1, auc, threshold, conf

In [103]:
def main_pipeline_autoencoder(dataset_name, optimizer_type):
    # Load Dataset
    if dataset_name == "kdd99":
        X_train = df_autoencoder_kdd99_train
        X_val = df_kdd99_validation
        X_test = df_kdd99_testing
        y_test = df_kdd99_testing_label
        y_val = df_kdd99_validation_label
    elif dataset_name == "unsw":
        X_train = df_autoencoder_unsw_train
        X_val = df_unsw_validation
        X_test = df_unsw_testing
        y_test = df_unsw_testing_label
        y_val = df_unsw_validation_label
   
    input_dim = X_train.shape[1]

    # Map indices to actual values
    activation_functions = ["relu", "tanh", "sigmoid"]
    output_activations = ["sigmoid", "tanh", "linear", "softmax", "relu"]
    loss_functions = ["mean_squared_error", "mean_absolute_error",
                      "binary_crossentropy", "categorical_crossentropy"]
    optimizers = ["Adam", "SGD", "Nadam"]

    if optimizer_type == "default":
        latent_dim = 32
        neurons_per_layer = 64
        learning_rate = 0.001
        dropout_rate = 0.2
        batch_size = 32
        epochs = 50
        activation_function = "relu"
        output_activation = "sigmoid"
        loss_function = "mean_squared_error"
        optimizer = "Adam"
        threshold_percentile = 95  # Default threshold percentile

    else:
        best_params = optimize_autoencoder(X_train, X_val, y_val, input_dim, optimizer_type)

        (latent_dim, neurons_per_layer, learning_rate, dropout_rate, batch_size, epochs, 
         activation_idx, output_activation_idx, loss_idx, optimizer_idx, threshold_percentile) = best_params

        activation_function = activation_functions[int(activation_idx)]
        output_activation = output_activations[int(output_activation_idx)]
        loss_function = loss_functions[int(loss_idx)]
        optimizer = optimizers[int(optimizer_idx)]

    # Train Final Model
    autoencoder = create_autoencoder(
        input_dim=input_dim,
        latent_dim=int(latent_dim),
        neurons_per_layer=int(neurons_per_layer),
        activation_function=activation_function,
        output_activation=output_activation,
        loss_function=loss_function,
        optimizer=optimizer,
        learning_rate=learning_rate,
        dropout_rate=dropout_rate,
    )

    autoencoder.fit(
        X_train,
        X_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        verbose=0,
    )

    # Evaluate
    accuracy, precision, recall, f1, auc, threshold, conf = evaluate_model(
        autoencoder, X_train, X_test, y_test, threshold_percentile)
    
    filename = dataset_name + "_" + optimizer_type + "_" + "autoencoder.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(autoencoder, f)

    print(f"\n=== For Dataset {dataset_name}, {optimizer_type} optimizer ===")
    print(f"Calculated Threshold Value: {threshold:.4f} (Percentile: {threshold_percentile:.1f}%)")

    print("\n=== Best Parameters ===")
    print(f"Latent Dim: {int(latent_dim)}")
    print(f"Neurons/Layer: {int(neurons_per_layer)}")
    print(f"Learning Rate: {learning_rate}")
    print(f"Dropout Rate: {dropout_rate}")
    print(f"Batch Size: {int(batch_size)}")
    print(f"Epochs: {int(epochs)}")
    print(f"Activation: {activation_function}")
    print(f"Output Activation: {output_activation}")
    print(f"Loss Function: {loss_function}")
    print(f"Optimizer: {optimizer}")
    print(f"Threshold Percentile: {threshold_percentile:.1f}%")

    print("\n=== Evaluation Metrics ===")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"confusion_matrix: \n{conf}")

#### ************Autoencoder************

In [104]:
main_pipeline_autoencoder('unsw', 'default')

1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 208us/step
2573/2573 ━━━━━━━━━━━━━━━━━━━━ 1s 239us/step

=== For Dataset unsw, default optimizer ===
Calculated Threshold Value: 1099280.9340 (Percentile: 95.0%)

=== Best Parameters ===
Latent Dim: 32
Neurons/Layer: 64
Learning Rate: 0.001
Dropout Rate: 0.2
Batch Size: 32
Epochs: 50
Activation: relu
Output Activation: sigmoid
Loss Function: mean_squared_error
Optimizer: Adam
Threshold Percentile: 95.0%

=== Evaluation Metrics ===
Accuracy: 0.4417
Precision: 0.2945
Recall: 0.0100
F1 Score: 0.0193
AUC: 0.4975
confusion_matrix: 
[[35917  1083]
 [44880   452]]


In [105]:
main_pipeline_autoencoder('kdd99', 'default')

2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 212us/step
3042/3042 ━━━━━━━━━━━━━━━━━━━━ 1s 208us/step

=== For Dataset kdd99, default optimizer ===
Calculated Threshold Value: 46.0779 (Percentile: 95.0%)

=== Best Parameters ===
Latent Dim: 32
Neurons/Layer: 64
Learning Rate: 0.001
Dropout Rate: 0.2
Batch Size: 32
Epochs: 50
Activation: relu
Output Activation: sigmoid
Loss Function: mean_squared_error
Optimizer: Adam
Threshold Percentile: 95.0%

=== Evaluation Metrics ===
Accuracy: 0.5546
Precision: 0.1458
Recall: 0.0115
F1 Score: 0.0214
AUC: 0.5578
confusion_matrix: 
[[53502  2777]
 [40572   474]]


#### ************Autoencoder + Whale Optimization Algorithm************

In [106]:
main_pipeline_autoencoder('kdd99', 'WOA')

2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 220us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 212us/step


2025/04/28 08:46:07 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.


2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 572us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 3s 917us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 758us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 233us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 211us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 208us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 266us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 242us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 254us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 227us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 251us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 248us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 243us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 275us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 239us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 742us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 589us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 29

2025/04/28 09:41:00 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: 0.9722709163346613, Global best: 0.9722709163346613, Runtime: 1857.57949 seconds


2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 213us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 227us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 306us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 227us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 234us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 204us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 212us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 283us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 298us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 212us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 219us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 237us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 253us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 279us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 283us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 203us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 218us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 215us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step


...

2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 320us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 306us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 896us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 714us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 306us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 256us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 653us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 823us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 724us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 854us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 282us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 342us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 236us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 545us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 646us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 311us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 258us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 739us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step


2025/04/29 02:29:59 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 19, Current best: 0.9276609305289993, Global best: 0.9276609305289993, Runtime: 1032.19961 seconds


2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 635us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 787us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 792us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 727us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 674us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 633us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 273us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 292us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 708us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 725us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 304us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 224us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 739us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 682us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 801us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 676us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 804us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 598us/step


2025/04/29 02:44:37 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 20, Current best: 0.9276609305289993, Global best: 0.9276609305289993, Runtime: 878.84595 seconds


2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 771us/step
3042/3042 ━━━━━━━━━━━━━━━━━━━━ 3s 829us/step

=== For Dataset kdd99, WOA optimizer ===
Calculated Threshold Value: 44.5299 (Percentile: 96.2%)

=== Best Parameters ===
Latent Dim: 52
Neurons/Layer: 128
Learning Rate: 0.0049271571804243835
Dropout Rate: 0.2330881002145331
Batch Size: 172
Epochs: 36
Activation: relu
Output Activation: softmax
Loss Function: binary_crossentropy
Optimizer: Adam
Threshold Percentile: 96.2%

=== Evaluation Metrics ===
Accuracy: 0.5701
Precision: 0.3747
Recall: 0.0289
F1 Score: 0.0537
AUC: 0.4734
confusion_matrix: 
[[54298  1981]
 [39859  1187]]


In [107]:
main_pipeline_autoencoder('unsw', 'WOA')

1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 202us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 221us/step


2025/04/29 02:49:22 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.


1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 613us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 730us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 304us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 232us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 221us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 530us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 465us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 274us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 268us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 339us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 257us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 267us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 243us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 511us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 498us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 265us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 241us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 252us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 246us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 327us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 410us/step
1750/1750 

2025/04/29 03:16:30 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: 0.8178772197609918, Global best: 0.8178772197609918, Runtime: 660.29125 seconds


1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 277us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 309us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 232us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 282us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 208us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 212us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 208us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 226us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 279us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 251us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 254us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 244us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 290us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 265us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 303us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 255us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 212us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 225us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 272us/step


...

1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 288us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 365us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 247us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 349us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 331us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 310us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 299us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 272us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 334us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 270us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 301us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 251us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 365us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 255us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 345us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 274us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 244us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 256us/step


2025/04/29 05:38:34 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 19, Current best: 0.34993352537114997, Global best: 0.34993352537114997, Runtime: 432.43831 seconds


1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 410us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 257us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 311us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 267us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 311us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 274us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 325us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 246us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 275us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 294us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 354us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 312us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 276us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 280us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 256us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 307us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 296us/step


2025/04/29 05:45:57 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 20, Current best: 0.34993352537114997, Global best: 0.34993352537114997, Runtime: 443.04995 seconds


1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 305us/step
2573/2573 ━━━━━━━━━━━━━━━━━━━━ 1s 283us/step

=== For Dataset unsw, WOA optimizer ===
Calculated Threshold Value: 934839.4767 (Percentile: 95.5%)

=== Best Parameters ===
Latent Dim: 46
Neurons/Layer: 111
Learning Rate: 0.003107311917064853
Dropout Rate: 0.20939858600612304
Batch Size: 233
Epochs: 32
Activation: tanh
Output Activation: linear
Loss Function: mean_squared_error
Optimizer: SGD
Threshold Percentile: 95.5%

=== Evaluation Metrics ===
Accuracy: 0.4530
Precision: 0.5344
Recall: 0.0508
F1 Score: 0.0928
AUC: 0.5625
confusion_matrix: 
[[34993  2007]
 [43028  2304]]


#### ************Autoencoder + Binary Differential Evolution Algorithm************

In [108]:
main_pipeline_autoencoder('kdd99', 'DE')

2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 505us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step


2025/04/29 05:48:06 AM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.


2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 546us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 657us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 285us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 234us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 232us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 205us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 323us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 334us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 275us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 239us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 221us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 213us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 268us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 235us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 232us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 292us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 407us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 254us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 255us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 231us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 218us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1

2025/04/29 06:37:27 AM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: 0.6567043618739903, Global best: 0.6567043618739903, Runtime: 1571.12730 seconds


2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 239us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 213us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 250us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 244us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 532us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 256us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 289us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 273us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 248us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 287us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 244us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 590us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 720us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 722us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 618us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 244us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 251us/step


...

2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 293us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 230us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 227us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 250us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 220us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 240us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 207us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 470us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 663us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 643us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 567us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 286us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 248us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 250us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 221us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 237us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 228us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 722us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 357us/step


2025/04/29 01:24:08 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 19, Current best: 0.1386632200886263, Global best: 0.1386632200886263, Runtime: 1476.94713 seconds


2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 281us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 232us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 301us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 224us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 241us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 252us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 283us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 253us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 596us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 883us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 707us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 543us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 238us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 216us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 2s 692us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step
2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 276us/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 257us/step


2025/04/29 01:49:32 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 20, Current best: 0.1386632200886263, Global best: 0.1386632200886263, Runtime: 1523.65923 seconds


2745/2745 ━━━━━━━━━━━━━━━━━━━━ 1s 239us/step
3042/3042 ━━━━━━━━━━━━━━━━━━━━ 1s 224us/step

=== For Dataset kdd99, DE optimizer ===
Calculated Threshold Value: 1380.2923 (Percentile: 95.3%)

=== Best Parameters ===
Latent Dim: 54
Neurons/Layer: 42
Learning Rate: 0.006761525524432734
Dropout Rate: 0.1573672040490052
Batch Size: 16
Epochs: 26
Activation: tanh
Output Activation: linear
Loss Function: mean_squared_error
Optimizer: Adam
Threshold Percentile: 95.3%

=== Evaluation Metrics ===
Accuracy: 0.5574
Precision: 0.1929
Recall: 0.0155
F1 Score: 0.0288
AUC: 0.7909
confusion_matrix: 
[[53610  2669]
 [40408   638]]


In [109]:
main_pipeline_autoencoder('unsw', 'DE')

1750/1750 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


2025/04/29 01:57:49 PM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.


1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 296us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 289us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 358us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 316us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 322us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 220us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 276us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 290us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 216us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 230us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 244us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 578us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 472us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 501us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 258us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step
1750/1750 

2025/04/29 02:34:55 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: 0.978406890015247, Global best: 0.978406890015247, Runtime: 1268.92026 seconds


1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 243us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 236us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 209us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 228us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 249us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 230us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 214us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 253us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 309us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 205us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 268us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 285us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 473us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 326us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 1s 449us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 1s 471us/step


...

1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 282us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 217us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 228us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 252us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 226us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 249us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 237us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 247us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 211us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 236us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 227us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 216us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 256us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 221us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step


2025/04/29 07:25:27 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 19, Current best: 0.8880250048837663, Global best: 0.8880250048837663, Runtime: 815.82525 seconds


1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 227us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 225us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 225us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 234us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 245us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 248us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 271us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 241us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 220us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 279us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 236us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 276us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 221us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 241us/step
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 0s 226us/step


2025/04/29 07:38:00 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 20, Current best: 0.8880250048837663, Global best: 0.8880250048837663, Runtime: 753.09813 seconds


1750/1750 ━━━━━━━━━━━━━━━━━━━━ 0s 237us/step
2573/2573 ━━━━━━━━━━━━━━━━━━━━ 1s 212us/step

=== For Dataset unsw, DE optimizer ===
Calculated Threshold Value: 25.1177 (Percentile: 96.5%)

=== Best Parameters ===
Latent Dim: 48
Neurons/Layer: 16
Learning Rate: 0.005141884102476518
Dropout Rate: 0.1
Batch Size: 133
Epochs: 100
Activation: relu
Output Activation: sigmoid
Loss Function: binary_crossentropy
Optimizer: Adam
Threshold Percentile: 96.5%

=== Evaluation Metrics ===
Accuracy: 0.4583
Precision: 0.6056
Recall: 0.0465
F1 Score: 0.0863
AUC: 0.5743
confusion_matrix: 
[[35628  1372]
 [43225  2107]]
